In [1]:
import pandas as pd
import os
import utm
import numpy as np
from tqdm import tqdm_notebook as tqdm
from scipy import spatial
from arcgis.gis import GIS
import arcgis.geocoding
import arcgis.geometry

#Create a map widget like you have done many times before
# gis = GIS("https://htm.esri.com/portal", "Mishkan", "password1")
# map = gis.map()

The data is initially loaded into a pandas dataframe, it is pickled and saved for speeding up restarts of this script

In [2]:
df_savepath = 'filter_df.df'
csv_zip_path ='../Data/AIS_2017_12_Zone11.zip'

try:
    df = pd.read_pickle(df_savepath)
except FileNotFoundError:
    df = pd.read_csv(csv_zip_path, infer_datetime_format=True, parse_dates=[1])

Initial content filters are given below and commented, remove or adjust any lines as needed

In [3]:
filtered_idx = np.all(np.vstack([
    df['VesselType'] != 52, # no tugs
    df['VesselType'] != 1023,
    df['VesselType'] != 1025,
    df['Status'] != 'moored', #nothing that is moored
    df['Status'] != 'at anchor', #nothing that is at anchor
    df['SOG'] >= 4 #no speeds less than 4 knots
                    ]), axis=0)

Filters are applied the dataframe is repickled

In [4]:
df = df[filtered_idx]
df.to_pickle(df_savepath)
print(f"The dataframe is reduced to only {len(df)} points")

The dataframe is reduced to only 910913 points


The following function is a quick way to detect interaction type

In [5]:
def detect_interaction(ship1, ship2):
    def norm_angle(ang):
        if ang < 0:
            ang += 360
        if ang > 360:
            ang -= 360
        return ang
    
    init1 = (ship1['LAT'].values[0], ship1['LON'].values[0])
    init2 = (ship2['LAT'].values[0], ship2['LON'].values[0])
    bear1 = ship1['COG'].values[0]
    bear2 = ship2['COG'].values[0]
    ship1_behind = norm_angle(bear1-180)
    ship2_behind = norm_angle(bear2-180)
    x1, y1, _, _ = utm.from_latlon(init1[0], init1[1])
    x2, y2, _, _ = utm.from_latlon(init2[0], init2[1])
    dx = x2 - x1
    dy = y2 - y1
    angle1 = 180*np.arctan2(dy, dx)/np.pi*-1+90
    angle2 = 180*np.arctan2(-dy, -dx)/np.pi*-1+90
    if (angle1 < ship1_behind+67.5 and angle1 > ship1_behind-67.5) or (angle2 < ship2_behind+67.5 and angle2 > ship2_behind-67.5):
        return 'Overtaking'
    elif (angle1 < bear1+10 and angle1 > bear1-10) or (angle2 < bear2+10 and angle2 > bear2-10):
        return 'Head-On'
    else:
        return 'Crossing'

In [6]:
time_start = df['BaseDateTime'].min()
time_delta = (df['BaseDateTime'].max()-df['BaseDateTime'].min()).total_seconds()/60
time_window = 30
time_step = 15
interactions_list = []
interaction_number = 0

for t in tqdm(np.arange(0, time_delta, time_step)):
    time_step_idx = np.all(np.vstack([
        df['BaseDateTime'] > time_start + pd.Timedelta(minutes=t),
        df['BaseDateTime'] < time_start + pd.Timedelta(minutes=t+time_window)
        ]
       ), axis=0)
    df_sub = df[time_step_idx]
    #Spatial distances are calculated
    distances = np.triu(spatial.distance.squareform(spatial.distance.pdist(df_sub.iloc[:, 2:4])))
    distances[distances > 0.067] = 0
    pairs = np.nonzero(distances)
    #Remove distances of ship relative to self
    MMSI_pairs = (df_sub['MMSI'].iloc[pairs[0]], df_sub['MMSI'].iloc[pairs[1]])
    ship_non_self_idx = np.argwhere(MMSI_pairs[0].values!=MMSI_pairs[1].values)
    ship_pairs = np.array([pairs[0][ship_non_self_idx], pairs[1][ship_non_self_idx]]).T[0]
    #Find each unique interaction
    mmsi_ship_pairs = df_sub['MMSI'].values[ship_pairs]
    if len(mmsi_ship_pairs) == 0:
        continue
    interactions = np.unique(mmsi_ship_pairs, axis=0)
    for pair in interactions:
        ship1 = df_sub[df_sub['MMSI'] == pair[0]]
        ship2 = df_sub[df_sub['MMSI'] == pair[1]]
        
        out = [ship1['MMSI'].values[0],
               ship2['MMSI'].values[0],
               np.mean([ship1['LAT'].values[0], ship2['LAT'].values[0]]),
               np.mean([ship1['LON'].values[0], ship2['LON'].values[0]]),
               ship1['BaseDateTime'].values[0],
               detect_interaction(ship1, ship2)
              ]
        interactions_list.append(out)

In [10]:
out_df = pd.DataFrame(interactions_list, columns=['MMSI1', 'MMSI2', 'LAT', 'LON', 'Timestamp', 'Interaction'])
out_df.to_csv('out.csv')